<a href="https://colab.research.google.com/github/MayaPikachu/Offense-detector/blob/main/Offense_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install unidecode
!pip install tokenizers

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from transformers import AutoTokenizer
from unidecode import unidecode
from tokenizers import ByteLevelBPETokenizer

In [ ]:
df = pd.read_csv("SBIC.v2.trn.csv")
dt = pd.read_csv("SBIC.v2.tst.csv")

In [ ]:
poststrn = df.post.values
resultstrn = df.offensiveYN.values

poststst = dt.post.values
resultstst = dt.offensiveYN.values

tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator([poststrn[:int(4E7)]], vocab_size=500, min_frequency=5)

vocab_size = tokenizer.get_vocab_size()

print(f"\n\n\tTokenizado! Vocab_size = {vocab_size}\n")

encode = lambda s: tokenizer.encode(s).ids
decode = lambda ids: tokenizer.decode(ids)



	Tokenizado! Vocab_size = 500



In [ ]:
block_size = 20
batch_size = 64
iter = 3000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def get_data(file, results, batch_size = batch_size):
  X, Y = [], []
  for i in range(batch_size):
    ix = torch.randint(0, len(file), (1, ))
    input = (encode(file[ix]) + [0] * block_size)[:block_size]
    X.append(input)
    Y.append(results[ix])
  return torch.tensor(X).float(), torch.nan_to_num(torch.tensor(Y).float(), nan=0.0)

get_data(poststrn, resultstrn)

(tensor([[158., 250., 100.,  ..., 260., 487., 295.],
         [ 49.,  51., 383.,  ...,  68., 284.,  72.],
         [ 34., 279., 257.,  ..., 316., 363.,  30.],
         ...,
         [ 35.,  68., 346.,  ..., 295.,  68., 329.],
         [ 45., 278.,  76.,  ..., 282., 420., 442.],
         [ 39., 305., 485.,  ..., 298., 257., 285.]]),
 tensor([0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.5000, 0.0000, 1.0000,
         1.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 1.0000,
         0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 1.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 1.0000, 1.0000, 1.0000,
         0.0000, 1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 1.0000, 1.0000, 1.0000,
         1.0000]))

In [ ]:
class MLP(nn.Module):
  def __init__(self, n_hidden):
    super().__init__()
    self.l1 = nn.Linear(block_size, n_hidden)
    self.l2 = nn.Linear(n_hidden, n_hidden)
    self.l3 = nn.Linear(n_hidden, 1)
    self.s = nn.Sigmoid()

  def forward(self, x):
    x = F.relu(self.l1(x))
    x = nn.Dropout(0.2)(F.relu(self.l2(x)))
    x = nn.Dropout(0.2)(self.l3(x))
    return self.s(x)



In [ ]:
model = MLP(128)
model.to(device)
params = list(model.parameters())
optimizer = torch.optim.Adam(params, lr=0.0003)
loss_fn = nn.BCELoss()
for i in range(iter):
  X, Y = get_data(poststrn, resultstrn)
  X = X.to(device)
  Y = Y.to(device)
  optimizer.zero_grad()
  logits = model(X)
  loss = loss_fn(logits, Y.unsqueeze(1))
  loss.backward()
  optimizer.step()
  if (i%100 == 0):
    print(f'step = {i}, loss = {loss}')


step = 0, loss = 34.46164321899414
step = 100, loss = 5.232851505279541
step = 200, loss = 1.0635298490524292
step = 300, loss = 0.7637502551078796
step = 400, loss = 0.8205668330192566
step = 500, loss = 0.6767224073410034
step = 600, loss = 0.6415577530860901
step = 700, loss = 0.7463389039039612
step = 800, loss = 0.6746700406074524
step = 900, loss = 0.6852728128433228
step = 1000, loss = 0.720058262348175
step = 1100, loss = 0.6961688995361328
step = 1200, loss = 0.6768128871917725
step = 1300, loss = 0.6743472814559937
step = 1400, loss = 0.6614426374435425
step = 1500, loss = 0.6822410821914673
step = 1600, loss = 0.6610668301582336
step = 1700, loss = 0.685003399848938
step = 1800, loss = 0.711157500743866
step = 1900, loss = 0.7497321963310242
step = 2000, loss = 0.6878183484077454
step = 2100, loss = 0.6482507586479187
step = 2200, loss = 0.6556747555732727
step = 2300, loss = 0.6547324657440186
step = 2400, loss = 0.693442702293396
step = 2500, loss = 0.6370221972465515
step

In [ ]:
precisao = 0.0
for i in range(30):
  X, Y = get_data(poststst, resultstst)
  X = X.to(device)
  Y = Y.to(device)
  logits = model(X)
  precisao += (logits.round() == Y.unsqueeze(1)).float().mean()/30.
  print(f'x: {decode(X[1].int().to("cpu").tolist())}, y = {Y[1].to("cpu")}, predito = {0 if logits[1].to("cpu").item() < .5 else 1}')

print(precisao)


x: Stupid kids of stupid liberals and st, y = 1.0, predito = 1
x: Society is like a box of chocolates, y = 1.0, predito = 1
x: NYC fiber optic cut on purpose..., y = 0.0, predito = 1
x: What’s the problem with replacing your dishw, y = 1.0, predito = 1
x: I would like to say and I can proudly say that there, y = 1.0, predito = 1
x: You're a girl.  You're not expect, y = 1.0, predito = 1
x: If you eat leftover pizza you are ince, y = 1.0, predito = 1
x: Run run as fast as you can Run run as fast as you can, y = 0.5, predito = 1
x: What do you call a Jewish pokémon train, y = 1.0, predito = 1
x: So I failed to get accepted at Harvar, y = 1.0, predito = 0
x: I have a hint for you in my usename @et_twe, y = 1.0, predito = 1
x: Today, I bought a Rolex from a black gu, y = 1.0, predito = 0
x: Women whose third child is born foll, y = 0.0, predito = 1
x: Aan indian was taking a dump and died. St, y = 1.0, predito = 1
x: what is the difference between a homosexu, y = 1.0, predito = 1
x: what d